In [0]:
def mount_adls(storage_account_name, container_name):
    # Get secrets from Key Vault
    clientid = dbutils.secrets.get(scope = 'f1secret', key = 'clientid')
    tenantid = dbutils.secrets.get(scope = 'f1secret', key = 'tenantid')
    clientsecret = dbutils.secrets.get(scope = 'f1secret', key = 'clientsecret')
    
    # Set spark configurations
    configs = {"fs.azure.account.auth.type": "OAuth",
              "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
              "fs.azure.account.oauth2.client.id": clientid,
              "fs.azure.account.oauth2.client.secret": clientsecret,
              "fs.azure.account.oauth2.client.endpoint": f"https://login.microsoftonline.com/{tenantid}/oauth2/token"}
    
    # Unmount the mount point if it already exists
    if any(mount.mountPoint == f"/mnt/{storage_account_name}/{container_name}" for mount in dbutils.fs.mounts()):
        dbutils.fs.unmount(f"/mnt/{storage_account_name}/{container_name}")
    
    # Mount the storage account container
    dbutils.fs.mount(
      source = f"abfss://{container_name}@{storage_account_name}.dfs.core.windows.net/",
      mount_point = f"/mnt/{storage_account_name}/{container_name}",
      extra_configs = configs)
    

In [0]:
mount_adls('f1pitstopdb1', 'raw')

In [0]:
mount_adls('f1pitstopdb1', 'process')

In [0]:
mount_adls('f1pitstopdb1', 'presentation')

In [0]:
display(dbutils.fs.mounts())